In [34]:
install.packages("bigsnpr")
install.packages("dplyr")
install.packages("readr")

library(bigsnpr)
library(dplyr)
library(readr)



The downloaded binary packages are in
	/var/folders/m5/0wg4jmc16qnb_0pxpgzhd_w80000gn/T//Rtmp0aR7S5/downloaded_packages

The downloaded binary packages are in
	/var/folders/m5/0wg4jmc16qnb_0pxpgzhd_w80000gn/T//Rtmp0aR7S5/downloaded_packages

The downloaded binary packages are in
	/var/folders/m5/0wg4jmc16qnb_0pxpgzhd_w80000gn/T//Rtmp0aR7S5/downloaded_packages


In [35]:
ref_prefix <- "1000G_20101123_v3_GIANT_chr1_23_minimacnamesifnotRS_CEU_MAF0.01"
gwas_file <- "gwas_for_clumping.txt"
rds_file <- paste0(ref_prefix, ".rds")
if (!file.exists(rds_file)) {
  snp_readBed(paste0(ref_prefix, ".bed"))
}

In [36]:
obj <- snp_attach(rds_file)
G <- obj$genotypes
map <- obj$map
names(map)[which(names(map) == "marker.ID")] <- "rsid"

In [37]:
gwas_df <- read_tsv(gwas_file, show_col_types = FALSE) %>%
  rename(rsid = SNP, chromosome = CHR, position = BP, pval = P)
  
S <- rep(NA, nrow(map))
matched <- match(map$rsid, gwas_df$rsid)
S[!is.na(matched)] <- -log10(gwas_df$pval[matched[!is.na(matched)]])

cat("GWAS SNPs matched to reference panel:", sum(!is.na(S)), "\n")

GWAS SNPs matched to reference panel: 7214 


In [38]:
ind_keep <- snp_clumping(
  G = G,
  infos.chr = map$chromosome,
  infos.pos = map$physical.pos,
  S = S,
  thr.r2 = 0.1,
  size = 250,
  ncores = 1
)

cat("Clumped SNPs retained:", length(ind_keep), "\n")


Clumped SNPs retained: 430409 


In [39]:
rsids_clumped <- map$rsid[ind_keep]
lead_snps <- semi_join(gwas_df, tibble(rsid = rsids_clumped), by = "rsid")

write_tsv(lead_snps, "clumped_snps.tsv")
cat("Saved final clumped summary to clumped_snps.tsv\n")

Saved final clumped summary to clumped_snps.tsv


In [40]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("biomaRt")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.1 (2024-06-14)

Warning message:
"package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'biomaRt'"
Old packages: 'BiocParallel', 'Matrix', 'R.cache', 'R.oo', 'RcppArmadillo',
  'cli', 'cluster', 'commonmark', 'curl', 'foreign', 'fs', 'ggplot2',
  'jsonlite', 'knitr', 'lattice', 'mgcv', 'mime', 'nlme', 'pbdZMQ', 'pillar',
  'pkgbuild', 'ps', 'rlang', 'sass', 'scales', 'stringi', 'tinytex', 'utf8',
  'xfun', 'xml2'



In [45]:
library(biomaRt)


df <- read.table("clumped_snps.tsv", header = TRUE) 
head(df)
snp_mart <- useEnsembl(biomart = "snp", dataset = "hsapiens_snp")

snp_info <- getBM(
  attributes = c("refsnp_id", "chr_name", "chrom_start", "allele"),
  filters = "snp_filter",
  values = df$rsid,
  mart = snp_mart
)

head(snp_info)


,rsid,chromosome,position,pval
,<chr>,<int>,<int>,<dbl>
1,rs301805,1,8481016,2.800013e-09
2,rs3001723,1,44037685,8.120822e-18
3,rs58097944,1,44054994,4.060037e-10
4,rs1343432,1,49396989,2.000000e-08
5,rs6669839,1,50625979,3.360006e-09
6,rs2186122,1,66470206,3.609940e-13


,refsnp_id,chr_name,chrom_start,allele
,<chr>,<chr>,<int>,<chr>
1,rs3001723,1,43572014,G/A
2,rs7555507,1,73300354,C/G/T
3,rs301805,1,8420956,T/C/G
4,rs1343432,1,48931317,G/C
5,rs12139556,1,73284243,T/C
6,rs12144768,1,72843322,G/A


In [ ]:
library(tidyr)
snp_info <- separate(snp_info, allele, into = c("ref", "alt"), sep = "/", remove = FALSE)

snp_info <- snp_info[!grepl("/", snp_info$alt), ]  

colnames(snp_info)[colnames(snp_info) == "refsnp_id"] <- "rsid"

df_annotated <- merge(df, snp_info[, c("rsid", "ref", "alt")], by = "rsid", all.x = TRUE)
write.table(df_annotated, "annotated_snps.tsv", sep = "\t", quote = FALSE, row.names = FALSE)


Warning message:
"Expected 2 pieces. Additional pieces discarded in 80 rows [2, 3, 7, 8, 13, 15,
16, 19, 20, 22, 23, 26, 28, 30, 31, 32, 33, 34, 35, 36, ...]."
